In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from dataload import *

import statsmodels.nonparametric.api as nparam
from statsmodels.nonparametric.kernel_regression import KernelReg
#from sklearn.model_selection import train_test_split

In [4]:
n = 1000
p = 0.9
a=1
b=1

In [5]:
df   = pd.DataFrame(np.random.uniform(0,1,size=(n, 3)),columns=['X1','X2','X3'])
df_reg = df.copy()

In [6]:
df_reg['Y'] = dataset(n,p,df_reg).h_df(a,b)

In [7]:
hsample_mse= [len(df_reg)]
col1 = ['sample']

for x in ['X1','X2','X3']:
    col1.append('$\textup{MSE} \left\{m_n \left('+f'${x[0]}_{x[1:]}$'+'\right) \right\}$')
    X_pred = np.linspace(0, 1, n)
    model= KernelReg(endog=[df_reg['Y'].to_numpy()],exog=[df_reg[x]],reg_type='lc',var_type='c', bw='cv_ls')
    Y_pred, _ = model.fit(X_pred)
    hsample_mse.append(round(np.mean((Y_pred-df_reg['Y'])**2),2))
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(X_pred, Y_pred, alpha=0.5,lw=3,c='purple')
    ax.plot(df[x],df_reg['Y'],'o',c='darkgrey',markersize=2)
    ax.set_xlabel(f'${x[0]}_{x[1:]}$')
    ax.set_ylabel('Y')
    plt.close(fig)
    fig.savefig('/Users/lilimatic/sobol_missing/images/regression/missing predictor mcar/H-Sample regression part '+str(x)+' in predictor.pdf',transparent=True,bbox_inches='tight')

print(hsample_mse)
print(col1)

[1000, 0.21, 0.18, 0.15]
['sample', '$\textup{MSE} \\left\\{m_n \\left($X_1$\right) \right\\}$', '$\textup{MSE} \\left\\{m_n \\left($X_2$\right) \right\\}$', '$\textup{MSE} \\left\\{m_n \\left($X_3$\right) \right\\}$']


In [8]:
cc_mse = []

for p in [0.9,0.75,0.4]:
    print(f'Analysis for p={p}')
    cc = dataset(n,p,df).mcar(a,b)
    cc['Y'] = dataset(len(cc),p,cc).h_df(a,b)
    #Fill list
    p_list = [len(cc)]
    ########################################
    X_pred = np.linspace(0, 1, len(cc))
    for x in ['X1','X2','X3']: 
        model_cc= KernelReg(endog=[cc['Y'].to_numpy()],exog=[cc[x]],reg_type='lc',var_type='c', bw='cv_ls')
        Y_pred_cc, _ = model_cc.fit(X_pred)
        p_list.append(np.mean((Y_pred_cc-cc['Y'])**2))
        #print(p_list)
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        plt.close(fig)
        ax.plot(X_pred, Y_pred_cc, alpha=0.5,lw=3,c='red')
        ax.plot(cc[x],cc['Y'],'o',c='black',markersize=2)
        ax.set_xlabel(f'${x[0]}_{x[1:]}$')
        ax.set_xlabel(f'${x[0]}_{x[1:]}$') 
        ax.set_ylabel('Y')
        ax.set_ylim((0,2))
    cc_mse.append(p_list)

cc_mse

#X1 [[76, 0.271408117552829], [49, 0.2986165008537631], [6, 0.13360902125670482]]
#X2 [[75, 0.2469377547132062],[49, 0.17031647383800377],[10, 0.21397161619318475]]

#X3 [[70, 0.1823793376986152], [42, 0.16863450861429116], [4, 0.2156401149004645]]

Analysis for p=0.9
Analysis for p=0.75
Analysis for p=0.4


[[743, 0.22700277356153767, 0.1924475668315899, 0.15343869090719658],
 [433, 0.2335695600335717, 0.19649306092801064, 0.15453043505817043],
 [69, 0.18070740581870876, 0.1665502153256783, 0.21400162615082544]]

In [10]:
mse_table = {'MSE / $\pi $':col1, 'H-Sample':hsample_mse}

tab1 = pd.DataFrame(mse_table)

a = dict(zip(['p=0.9','p=0.75','p=0.4'],cc_mse))

tab2 = pd.DataFrame(a)

full = pd.concat([tab1,tab2],axis=1)

#full
#print(full.to_latex(index=False,float_format="{:.2f}".format))